# Animal Classification Model Training
---

## 1. Import Required Libraries

In [ ]:
from tensorflow.keras.applications import EfficientNetB3
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, CSVLogger
from utils import load_dataset
import nbimporter
from data_preprocessing import dataset_path

---

## 2. Load Training and Validation Data

In [ ]:
train_data, val_data = load_dataset(dataset_path)  # Load the training and validation datasets

---

## 3. Build Model Architecture

In [ ]:
# Load EfficientNetB3 base (pre-trained on ImageNet)
base_model = EfficientNetB3(
    weights='imagenet',
    include_top=False,
    input_shape=(224, 224, 3)
)

# Unfreeze base model initially
base_model.trainable = True

x = base_model.output
x = GlobalAveragePooling2D()(x)

# First dense block
x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)

# Second dense block
x = Dense(256, activation='relu')(x)
x = BatchNormalization()(x)

# Output layer
predictions = Dense(10, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

---

## 4. Compile the Model

In [ ]:
# Compile
model.compile(
    optimizer=Adam(learning_rate=1e-5),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

---

## 5. Configure Training Callbacks

In [ ]:
# Early stopping to prevent overfitting
early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True) 

# Checkpoint to save the best model based on validation accuracy
checkpoint = ModelCheckpoint("/Outputs/best_model.h5", save_best_only=True, monitor='val_accuracy')

# CSV Logger to log training history each epoch
csv_logger = CSVLogger('/Outputs/training_log.csv', append=False)

---

## 6. Train the Model

In [ ]:
# Train the model
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=50,
    callbacks=[early_stop, checkpoint, csv_logger]
)

---

## 7. Visualize Training Results

In [ ]:
from utils import display_Training_History

# Display training history (accuracy and loss) for both training and validation sets
display_Training_History()

---